## Importação das bibliotecas

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## Carregar imagem

In [ ]:
original = cv2.imread('raiox-pulmao.png', cv2.IMREAD_GRAYSCALE)

## Aplicação do filtro gaussiano

In [ ]:
img_filtered = cv2.GaussianBlur(original, (5, 5), 0)

## Limiarização de Otsu

In [ ]:
ret, img_thresh = cv2.threshold(img_filtered, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

## Detecção de bordas com Canny

In [ ]:
img_edges = cv2.Canny(img_thresh, 100, 200)

# Dilatação para juntar bordas desconectadas
kernel = np.ones((3, 3), np.uint8)
img_dilated = cv2.dilate(img_edges, kernel, iterations=2)

## Detecção dos contornos dos pulmões

In [ ]:
contours, hierarchy = cv2.findContours(img_dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Selecionar os dois maiores contornos (os pulmões)
areas = [cv2.contourArea(c) for c in contours]
sorted_areas = sorted(areas, reverse=True)
largest_contours = sorted_areas[:2]
lung_mask = np.zeros_like(img_dilated)

for cnt in contours:
    if cv2.contourArea(cnt) in largest_contours:
        cv2.drawContours(lung_mask, [cnt], -1, 255, -1)

# Aplicação da máscara para extrair apenas os pulmões
lung = cv2.bitwise_and(original, original, mask=lung_mask)

## Exibição dos resultados

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
axs[0, 0].imshow(original, cmap='gray')
axs[0, 0].set_title('Imagem original')
axs[0, 1].imshow(img_filtered, cmap='gray')
axs[0, 1].set_title('Imagem após filtro gaussiano')
axs[1, 0].imshow(img_thresh, cmap='gray')
axs[1, 0].set_title('Imagem binarizada com Otsu')
axs[1, 1].imshow(lung, cmap='gray')
axs[1, 1].set_title('Segmentação dos pulmões')
plt.show()